In [56]:
import kagglehub
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [57]:
path = kagglehub.dataset_download("jsphyg/weather-dataset-rattle-package")
print("Path to dataset files:", path)
filepath = path + "/weatherAUS.csv"
print("Path to main dataset:", filepath)
dataset = pd.read_csv(filepath)

Using Colab cache for faster access to the 'weather-dataset-rattle-package' dataset.
Path to dataset files: /kaggle/input/weather-dataset-rattle-package
Path to main dataset: /kaggle/input/weather-dataset-rattle-package/weatherAUS.csv


# 1. Размер датасета и количество признаков

In [58]:
dataset.shape

(145460, 23)

# 2. Наличие пропущенных значений

In [59]:
print(dataset.isnull().sum().sum() / dataset.size * 100)

10.259745694319072


# 3. Несбалансированные классы, соотношение

In [60]:
dataset['RainToday'].value_counts(normalize=True).sort_index() * 100

,proportion
RainToday,
No,77.580714
Yes,22.419286


На сайте указано как 76/22/2 ввиду наличия пропущенных значений. Похоже, что value_counts закинул NA в No, так что в данном случае я беру информацию с kaggle.

# 4. Наличие выбросов и их процент

In [61]:
def quantile(column_name):
  Q1 = dataset[column_name].quantile(0.25)
  Q3 = dataset[column_name].quantile(0.75)
  IQR = Q3 - Q1
  lower_bound = Q1 - 1.5 * IQR
  upper_bound = Q3 + 1.5 * IQR
  outsiders = dataset[(dataset[column_name] < lower_bound) | (dataset[column_name] > upper_bound)]
  return len(outsiders)


number_columns = dataset.select_dtypes(include=['float64', 'int64']).columns
for col in number_columns:
  outsiders_count = quantile(col)
  total_count = len(dataset[col].dropna())
  percentage = round(outsiders_count / total_count * 100, 2)
  print(col, percentage)

MinTemp 0.04
MaxTemp 0.34
Rainfall 17.99
Evaporation 2.41
Sunshine 0.0
WindGustSpeed 2.29
WindSpeed9am 1.26
WindSpeed3pm 1.77
Humidity9am 1.0
Humidity3pm 0.0
Pressure9am 0.91
Pressure3pm 0.7
Cloud9am 0.0
Cloud3pm 0.0
Temp9am 0.18
Temp3pm 0.54


Насколько я поняла, надо выбрать максимальный выброс из всех числовых столбцов. В моем случае наибольший выброс это 18% в столбце Rainfall (это количество осадков, записанных в конкретную дату в мм)

# 5. Количество числовых признаков

In [62]:
len(dataset.select_dtypes(include=['float64', 'int64']).columns)


16

# 6. Количество категориальных признаков

In [64]:
dataset['Date'] = pd.to_datetime(dataset['Date'], errors='coerce', format="%Y-%m-%d")

print(len(dataset.select_dtypes(include=['category', 'object', 'bool']).columns))
print(len(dataset.select_dtypes(include=['datetime']).columns))
dataset['Date']

6
1


,Date
0,2008-12-01
1,2008-12-02
2,2008-12-03
3,2008-12-04
4,2008-12-05
...,...
145455,2017-06-21
145456,2017-06-22
145457,2017-06-23
145458,2017-06-24
